## Dog Breed Classification

In this project we will use traditional CNN, CNN with data augmentation and finally transfer Learning by VGG16 model with weights pre-trained on Imagenet to solve the dog breed classification problem

### Load Dataset Files

In [0]:
#Import necessary libraries to fetch the train and test data...
import os
from zipfile import ZipFile
import cv2
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn import metrics
from sklearn.metrics import classification_report

In [3]:
%tensorflow_version 2.x
import tensorflow as tf
from tensorflow import keras
tf.random.set_seed(42)
print(tf.__version__)

TensorFlow 2.x selected.
2.1.0


In [0]:
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPool2D
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import Reshape
from tensorflow.keras.layers import Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.layers import LeakyReLU
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img, img_to_array, array_to_img

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


Now, upload the given dataset file shared with you in your google drive and give its path for the below given `project_path` variable. For example, a path is given below according to the file path in our google drive. You need to change this to match the path of yours.

In [0]:
project_path = "/content/drive/My Drive/Great_Lakes_Assignments/10_ComputerVision_CNN_Project2/DogBreed_Classification/"

Run the below code to extract all the images in the train.zip files given in the dataset. We are going to use these images as train and validation sets and their labels in further steps.

In [0]:
from zipfile import ZipFile
with ZipFile(project_path+'train.zip', 'r') as z:
  z.extractall()

Repeat the same step for test.zip

In [0]:
with ZipFile(project_path+'test.zip', 'r') as z:
  z.extractall()

Repeat the same step for sample_submission.csv.zip

In [0]:
with ZipFile(project_path+'sample_submission.csv.zip', 'r') as z:
  z.extractall()

Repeat the same step for labels.csv.zip

In [0]:
with ZipFile(project_path+'labels.csv.zip', 'r') as z:
  z.extractall()

After this process, we will have 4 files - Train folder, test folder and labels.csv and sample_submission.csv as part of your google drive

### Read labels.csv file using pandas

In [0]:
df_labels = pd.read_csv('/content/labels.csv')

In [12]:
df_labels.head()

,id,breed
0,000bec180eb18c7604dcecc8fe0dba07,boston_bull
1,001513dfcb2ffafc82cccf4d8bbaba97,dingo
2,001cdf01b096e06d78e9e5112d419397,pekinese
3,00214f311d5d2247d5dfe4fe24b2303d,bluetick
4,0021f9ceb3235effd7fcde7f7538ed62,golden_retriever


### Print the count of each category of Dogs given in the dataset



In [13]:
print('Total count of dogs: ', df_labels.shape[0])
print('Number of dog Category: ', len(df_labels.breed.unique()))

Total count of dogs:  10222
Number of dog Category:  120


In [14]:
print('Count of each category of Dogs:')
df_labels.groupby('breed').size()

Count of each category of Dogs:


breed
affenpinscher                      80
afghan_hound                      116
african_hunting_dog                86
airedale                          107
american_staffordshire_terrier     74
                                 ... 
welsh_springer_spaniel             79
west_highland_white_terrier        81
whippet                            95
wire-haired_fox_terrier            82
yorkshire_terrier                  82
Length: 120, dtype: int64

### Get one-hot encodings of labels

In [0]:
#Get lable encoding for labels

from sklearn import preprocessing

le = preprocessing.LabelEncoder()
le.fit(df_labels.breed)
df_labels['breedEncoded'] = le.transform(df_labels.breed)

In [16]:
df_labels.head()

,id,breed,breedEncoded
0,000bec180eb18c7604dcecc8fe0dba07,boston_bull,19
1,001513dfcb2ffafc82cccf4d8bbaba97,dingo,37
2,001cdf01b096e06d78e9e5112d419397,pekinese,85
3,00214f311d5d2247d5dfe4fe24b2303d,bluetick,15
4,0021f9ceb3235effd7fcde7f7538ed62,golden_retriever,49


In [0]:
labels = tf.keras.utils.to_categorical(df_labels.breedEncoded)

In [18]:
labels

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [19]:
labels.shape

(10222, 120)

## Preparing training dataset
1. Write a code which reads each and every id from labels.csv file and loads the corresponding image (in RGB - 128, 128, 3) from the train folder. <br>
2. Create 2 variables <br> 
     a.  x_train - Should have all the images of the dogs from train folder <br>
     b.  y_train - Corresponding label of the dog <br>
<u>Note:</u> The id of the dog images and its corresponding labels are available in labels.csv file   
<u>Hint:</u> Watch the video shared on "Preparing the training dataset" if you face issue on creating the training dataset

In [0]:
X_train = []

ImgCnt = df_labels.shape[0]

for i in range(ImgCnt):
  try:
      dummy = cv2.imread('/content/train/' + df_labels.id[i] + '.jpg')
      dummy = cv2.resize(dummy,(128,128)) #resize to have all the images of same size
      X_train.append(dummy)
  except Exception as e:
      print(e)

In [21]:
print('x_train Count: ', len(X_train))

x_train Count:  10222


Normalize the training data and convert into 4 dimensions so that it can be used as an input to conv layers in the model

In [0]:
#Convert the list to numpy array for easy manipulation...
X_train_arr = np.asarray(X_train)

In [0]:
X_train_std = X_train_arr/255

In [24]:
X_train_std.shape

(10222, 128, 128, 3)

In [0]:
X_train_std = X_train_std.reshape(X_train_std.shape[0], 128, 128, 3).astype('float32')

In [26]:
X_train_std.shape

(10222, 128, 128, 3)

### Split the training and validation data from `x_train_data` and `y_train_data` obtained from above step

In [0]:
from sklearn.model_selection import train_test_split

X_train_split, X_val_split, y_train_split, y_val_split = train_test_split(X_train_std, labels, test_size=0.2, random_state=2)


In [28]:
print ("No. of images in train dataset: ", len(X_train_split))
print ("No. of images in Validation dataset: ", len(X_val_split))

No. of images in train dataset:  8177
No. of images in Validation dataset:  2045


### Loading the test data
Read the id column from the samples_submission.csv and store it in test_img

In [0]:
test_img = pd.read_csv('/content/sample_submission.csv')['id']

In [30]:
test_img.head()

0    000621fb3cbb32d8935728e48679680e
1    00102ee9d8eb90812350685311fe5890
2    0012a730dfa437f5f3613fb75efcd4ce
3    001510bc8570bbeee98c8d80c8a95ec1
4    001a5f3114548acdefa3d4da05474c2e
Name: id, dtype: object

In [31]:
test_img.shape

(10357,)

Run the below code to load the test image files in x_test_feature

In [0]:
x_test_feature = []
ImgCnt = len(test_img)

for i in range(ImgCnt):
  try:
      dummy = cv2.imread('/content/test/' + test_img[i] + '.jpg')
      dummy = cv2.resize(dummy,(128,128)) #resize to have all the images of same size
      x_test_feature.append(dummy)
  except Exception as e:
      print(e)

In [33]:
print('x_test_feature Count: ', len(x_test_feature))

x_test_feature Count:  10357


Normalize the test data and convert it into 4 dimensions

In [0]:
#Convert the list to numpy array for easy manipulation...
X_test_arr = np.asarray(x_test_feature)

In [0]:
X_test_std = X_test_arr/255

In [36]:
X_test_std.shape

(10357, 128, 128, 3)

In [0]:
X_test_std = X_test_std.reshape(X_test_std.shape[0], 128, 128, 3).astype('float32')

In [38]:
X_test_std.shape

(10357, 128, 128, 3)

### Build a basic conv neural network with 2 conv layers (kernel sizes - 5 and 3) add layers as mentioned below for classification.

1. Add a Dense layer with 256 neurons with `relu` activation

2. Add a Dense layer with 120 neurons as final layer (as there are 120 classes in the given dataset) with `softmax` activation for classifiaction. 

In [0]:
model = Sequential()
model.add(Conv2D(filters=32, input_shape = (128,128,3), kernel_size=5)) 
model.add(Conv2D(filters=64, kernel_size=3))

model.add(Flatten()) 

# fully connected layer
model.add(Dense(units=256, kernel_initializer = 'he_normal', activation = 'relu'))
model.add(Dense(units = 120, activation = 'softmax'))

In [0]:
model.compile(optimizer='adam', loss = 'categorical_crossentropy',metrics = ['accuracy'])

In [0]:
#Saving the best model using model checkpoint callback
model1_checkpoint=tf.keras.callbacks.ModelCheckpoint('model1.h5',save_best_only=True, monitor='val_accuracy', mode='max', verbose=1)

### Use batch_size = 128 and epochs = 10 and execute the model

In [42]:
Model_1 = model.fit(X_train_split, y_train_split,
                    epochs=10, 
                    validation_data=(X_val_split, y_val_split),
                    verbose = 1,
                    batch_size=128, callbacks = [model1_checkpoint])

Train on 8177 samples, validate on 2045 samples
Epoch 1/10
8064/8177 [============================>.] - ETA: 1s - loss: 13.2790 - accuracy: 0.0098
Epoch 00001: val_accuracy improved from -inf to 0.01027, saving model to model1.h5
8177/8177 [==============================] - 115s 14ms/sample - loss: 13.1617 - accuracy: 0.0098 - val_loss: 4.7878 - val_accuracy: 0.0103
Epoch 2/10
8064/8177 [============================>.] - ETA: 1s - loss: 4.7860 - accuracy: 0.0089
Epoch 00002: val_accuracy did not improve from 0.01027
8177/8177 [==============================] - 92s 11ms/sample - loss: 4.7864 - accuracy: 0.0089 - val_loss: 4.7888 - val_accuracy: 0.0103
Epoch 3/10
8064/8177 [============================>.] - ETA: 1s - loss: 4.7831 - accuracy: 0.0124
Epoch 00003: val_accuracy did not improve from 0.01027
8177/8177 [==============================] - 94s 12ms/sample - loss: 4.7832 - accuracy: 0.0124 - val_loss: 4.7865 - val_accuracy: 0.0103
Epoch 4/10
8064/8177 [============================>

#The model accuracy is very poor !!!!

### Use Data Augmentation in the above model to see if the accuracy improves


In [43]:
from keras.preprocessing.image import ImageDataGenerator

Using TensorFlow backend.


In [0]:
train_datagen = ImageDataGenerator( rotation_range=90,
                 width_shift_range=0.1, height_shift_range=0.1,
                 horizontal_flip=True)
train_datagen.fit(X_train_split)

In [0]:
val_datagen = ImageDataGenerator( rotation_range=90,
                 width_shift_range=0.1, height_shift_range=0.1,
                 horizontal_flip=True)
val_datagen.fit(X_val_split)

### Using the above objects, create the image generators with variable names `train_generator` and `val_generator`

You need to use train_datagen.flow() and val_datagen.flow()

In [0]:
train_generator = train_datagen.flow(X_train_split, y_train_split, batch_size=10)
val_generator = val_datagen.flow(X_val_split, y_val_split, batch_size=10)

### Fit the model using fit_generator() using `train_generator` and `val_generator` from the above step with 10 epochs

In [78]:
model.fit_generator(train_generator,\
                    epochs=10, steps_per_epoch= X_train_split.shape[0]//32, \
                    verbose=1,validation_data=val_generator, validation_steps = X_val_split.shape[0]//32)

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 255 steps, validate for 63 steps
Epoch 1/10
255/255 [==============================] - 93s 366ms/step - loss: 4.7752 - accuracy: 0.0114 - val_loss: 4.7888 - val_accuracy: 0.0111
Epoch 2/10
255/255 [==============================] - 93s 365ms/step - loss: 4.7742 - accuracy: 0.0090 - val_loss: 4.7893 - val_accuracy: 0.0111
Epoch 3/10
255/255 [==============================] - 93s 366ms/step - loss: 4.7755 - accuracy: 0.0094 - val_loss: 4.7899 - val_accuracy: 0.0111
Epoch 4/10
255/255 [==============================] - 91s 359ms/step - loss: 4.7751 - accuracy: 0.0125 - val_loss: 4.7902 - val_accuracy: 0.0159
Epoch 5/10
255/255 [==============================] - 93s 365ms/step - loss: 4.7772 - accuracy: 0.0102 - val_loss: 4.7902 - val_accuracy: 0.0159
Epoch 6/10
255/255 [==============================] - 94s 367ms/step - loss: 4.7771 - accuracy: 0.0149 - val_loss: 4.7896 - val_accuracy: 0.0159
Epoch 7/10
255/255 [=================

# Model accuracy is still poor!!!

### Lets use Transfer Learning

Download the vgg wieght file from here : https://github.com/MinerKasch/applied_deep_learning/blob/master/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5

In [0]:
project_path = "/content/drive/My Drive/Great_Lakes_Assignments/10_ComputerVision_CNN_Project2/vgg16_weights_tf_dim_ordering_tf_kernels_notop.h5"

Use the below code to load VGG16 weights trained on ImageNet

In [0]:
#Define some parameters
img_size = 128
img_depth = 3  

In [0]:
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
# Instantiate the model with the pre-trained weights (no top)
base_model= VGG16(include_top=False, input_shape=(img_size,img_size, img_depth)
                 , pooling='avg',weights=project_path)

Print the summary of the base_model

In [84]:
base_model.summary()

Model: "vgg16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 128, 128, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 128, 128, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 128, 128, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 64, 64, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 64, 64, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 64, 64, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 32, 32, 128)       0     

In [91]:
base_model.output

<tf.Tensor 'global_average_pooling2d_1/Identity:0' shape=(None, 512) dtype=float32>

### Make all the layers in the base_model (VGG16) to be non-trainable

In [0]:
#Set pre-trained model layers to not trainable
for layer in base_model.layers:
    layer.trainable = False

### Add the following classification layers to the imported VGG Model <br>
1. Flatten Layer
2. Dense layer with 1024 neurons with activation as Relu
3. Dense layer with 256 neurons with activation as Relu
4. Dense layer with 120 neurons with activation as Softmax

In [0]:
#get Output layer of Pre0trained model
x = base_model.output

x = Flatten()(x)
x = Dense(units=1024, activation = 'relu')(x)
x = Dense(units=256, activation = 'relu')(x)
#Add output layer
prediction = Dense(units = 120, activation = 'softmax')(x)


In [0]:
#Using Keras Model class
final_model = tf.keras.models.Model(inputs=base_model.input, #Pre-trained model input as input layer
                                    outputs=prediction) #Output layer added

In [0]:
final_model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [99]:
final_model.summary()

Model: "model_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_2 (InputLayer)         [(None, 128, 128, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 128, 128, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 128, 128, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 64, 64, 64)        0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 64, 64, 128)       73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 64, 64, 128)       147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 32, 32, 128)       0   

In [0]:
batchsize=128

### Fit and compile the model with batch_size = 128 and epochs = 10 and execute the model

Try to get training and validation accuracy to be more than 90%

In [0]:
modelfinal_checkpoint = tf.keras.callbacks.ModelCheckpoint('modelfinal.h5',save_best_only=True, monitor='val_accuracy', mode='max', verbose=1)

In [106]:
Model_Final = final_model.fit(X_train_split, y_train_split,
                    epochs=10, 
                    validation_data=(X_val_split, y_val_split),
                    verbose = 1,
                    batch_size=128,callbacks = [modelfinal_checkpoint])

Train on 8177 samples, validate on 2045 samples
Epoch 1/10
8064/8177 [============================>.] - ETA: 1s - loss: 3.7423 - accuracy: 0.1210
Epoch 00001: val_accuracy improved from -inf to 0.12078, saving model to modelfinal.h5
8177/8177 [==============================] - 105s 13ms/sample - loss: 3.7411 - accuracy: 0.1216 - val_loss: 3.8008 - val_accuracy: 0.1208
Epoch 2/10
8064/8177 [============================>.] - ETA: 1s - loss: 3.5770 - accuracy: 0.1500
Epoch 00002: val_accuracy improved from 0.12078 to 0.12861, saving model to modelfinal.h5
8177/8177 [==============================] - 103s 13ms/sample - loss: 3.5738 - accuracy: 0.1504 - val_loss: 3.7062 - val_accuracy: 0.1286
Epoch 3/10
8064/8177 [============================>.] - ETA: 1s - loss: 3.4604 - accuracy: 0.1670
Epoch 00003: val_accuracy improved from 0.12861 to 0.13888, saving model to modelfinal.h5
8177/8177 [==============================] - 104s 13ms/sample - loss: 3.4611 - accuracy: 0.1674 - val_loss: 3.6131 

In [107]:
##As we can see the model accurracy is increasing so we can run it for more epochs
Model_Final = final_model.fit(X_train_split, y_train_split,
                    epochs=20, 
                    validation_data=(X_val_split, y_val_split),
                    verbose = 1,
                    batch_size=128,callbacks = [modelfinal_checkpoint],initial_epoch=10)

Train on 8177 samples, validate on 2045 samples
Epoch 11/20
8064/8177 [============================>.] - ETA: 1s - loss: 2.8175 - accuracy: 0.2899
Epoch 00011: val_accuracy improved from 0.19413 to 0.19902, saving model to modelfinal.h5
8177/8177 [==============================] - 102s 13ms/sample - loss: 2.8187 - accuracy: 0.2900 - val_loss: 3.2832 - val_accuracy: 0.1990
Epoch 12/20
8064/8177 [============================>.] - ETA: 1s - loss: 2.7616 - accuracy: 0.3011
Epoch 00012: val_accuracy improved from 0.19902 to 0.20831, saving model to modelfinal.h5
8177/8177 [==============================] - 100s 12ms/sample - loss: 2.7567 - accuracy: 0.3007 - val_loss: 3.2345 - val_accuracy: 0.2083
Epoch 13/20
8064/8177 [============================>.] - ETA: 1s - loss: 2.6962 - accuracy: 0.3155
Epoch 00013: val_accuracy improved from 0.20831 to 0.20978, saving model to modelfinal.h5
8177/8177 [==============================] - 101s 12ms/sample - loss: 2.6977 - accuracy: 0.3153 - val_loss: 3

In [0]:
train_generator = train_datagen.flow(X_train_split, y_train_split, batch_size=128)
val_generator = val_datagen.flow(X_val_split, y_val_split, batch_size=128)

In [0]:
modelfinal1_checkpoint = tf.keras.callbacks.ModelCheckpoint('modelfinal1.h5',save_best_only=True, monitor='val_accuracy', mode='max', verbose=1)

In [114]:
final_model.fit_generator(train_generator,\
                    epochs=10, steps_per_epoch=X_train_split.shape[0]//batchsize, \
                    verbose=1,validation_data=val_generator, validation_steps = X_val_split.shape[0]//batchsize,callbacks = [modelfinal1_checkpoint])

  ...
    to  
  ['...']
  ...
    to  
  ['...']
Train for 63 steps, validate for 15 steps
Epoch 1/10
62/63 [============================>.] - ETA: 1s - loss: 3.2275 - accuracy: 0.2180
Epoch 00001: val_accuracy improved from -inf to 0.15469, saving model to modelfinal1.h5
63/63 [==============================] - 106s 2s/step - loss: 3.2276 - accuracy: 0.2182 - val_loss: 3.6595 - val_accuracy: 0.1547
Epoch 2/10
62/63 [============================>.] - ETA: 1s - loss: 3.2086 - accuracy: 0.2264
Epoch 00002: val_accuracy did not improve from 0.15469
63/63 [==============================] - 106s 2s/step - loss: 3.2099 - accuracy: 0.2272 - val_loss: 3.6300 - val_accuracy: 0.1521
Epoch 3/10
62/63 [============================>.] - ETA: 1s - loss: 3.1626 - accuracy: 0.2358
Epoch 00003: val_accuracy improved from 0.15469 to 0.16563, saving model to modelfinal1.h5
63/63 [==============================] - 106s 2s/step - loss: 3.1598 - accuracy: 0.2365 - val_loss: 3.6135 - val_accuracy: 0.1656
Ep